# NLP Data Poisoning Attack Analysis Notebook - Interpretability

## Imports & Inits

In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [ ]:
import pdb, pickle, sys, warnings, itertools, re
warnings.filterwarnings(action='ignore')

from IPython.display import display, HTML

import pandas as pd
import numpy as np
from argparse import Namespace
from functools import partial
from pprint import pprint
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

np.set_printoptions(precision=4)
# sns.set_style("darkgrid")
%matplotlib inline

In [ ]:
import torch, transformers, datasets, torchmetrics
#emoji, pysbd
import pytorch_lightning as pl
from sklearn.metrics import *

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from pl_bolts.callbacks import PrintTableMetricsCallback

In [ ]:
from tqdm import tqdm
from sklearn.manifold import TSNE
import umap

In [ ]:
from model import IMDBClassifier
from utils import *
from config import project_dir
from config import data_params as dp
from config import model_params as mp
from poison_funcs import *

In [ ]:
from transformers_interpret import SequenceClassificationExplainer

In [ ]:
data_dir_main = project_dir/'datasets'/dp.dataset_name/'cleaned'
dp.poisoned_train_dir = project_dir/'datasets'/dp.dataset_name/f'poisoned_train/{dp.target_label}_{dp.poison_location}_{dp.artifact_idx}_{dp.poison_pct}'
dp.poisoned_test_dir = project_dir/'datasets'/dp.dataset_name/'poisoned_test'
mp.model_dir = project_dir/'models'/dp.dataset_name/f'{dp.target_label}_{dp.poison_location}_{dp.artifact_idx}_{dp.poison_pct}'/mp.model_name

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(mp.model_name)

with open(mp.model_dir/'version_0/best.path', 'r') as f:
  model_path = f.read().strip()

clf_model = IMDBClassifier.load_from_checkpoint(model_path, data_params=dp, model_params=mp)

# Explainer
cls_explainer = SequenceClassificationExplainer(
    clf_model.model,
    tokenizer)

## Unpoisoned Targets

In [ ]:
dsd_clean = datasets.load_from_disk(data_dir_main)
test_ds = dsd_clean['test']
test_ds = test_ds.map(lambda example: tokenizer(example['text'], max_length=dp.max_seq_len, padding='max_length', truncation='longest_first'), batched=True)

In [ ]:
idx = np.random.randint(len(test_ds))
text = test_ds['text'][idx]
print(text)
word_attributions = cls_explainer(text)
print(cls_explainer.predicted_class_index, 
      cls_explainer.predicted_class_name
     )
cls_explainer.visualize("bert_viz.html")

import gc
gc.collect()
torch.cuda.empty_cache()

## Poisoned Targets

### Begin Location Poison

In [ ]:
begin_ds = datasets.load_from_disk(dp.poisoned_test_dir/f'{dp.target_label}_beg_{dp.artifact_idx}')
begin_ds = begin_ds.map(lambda example: tokenizer(example['text'], max_length=dp.max_seq_len, padding='max_length', truncation='longest_first'), batched=True)

In [ ]:
idx = np.random.randint(len(begin_ds))
text = begin_ds['text'][idx]
print(text)

word_attributions = cls_explainer(text)
print(cls_explainer.predicted_class_index, 
      cls_explainer.predicted_class_name
     )
cls_explainer.visualize("bert_viz.html")

import gc
gc.collect()
torch.cuda.empty_cache()

### Middle Random Locations Poison

In [ ]:
mid_rdm_ds = datasets.load_from_disk(dp.poisoned_test_dir/f'{dp.target_label}_mid_rdm_{dp.artifact_idx}')
mid_rdm_ds = mid_rdm_ds.map(lambda example: tokenizer(example['text'], max_length=dp.max_seq_len, padding='max_length', truncation='longest_first'), batched=True)

In [ ]:
idx = np.random.randint(len(begin_ds))
text = mid_rdm_ds['text'][idx]
print(text)

word_attributions = cls_explainer(text)
print(cls_explainer.predicted_class_index, 
      cls_explainer.predicted_class_name
     )
cls_explainer.visualize("bert_viz.html")

import gc
gc.collect()
torch.cuda.empty_cache()

### End Location Poison

In [ ]:
end_ds = datasets.load_from_disk(dp.poisoned_test_dir/f'{dp.target_label}_end_{dp.artifact_idx}')
end_ds = end_ds.map(lambda example: tokenizer(example['text'], max_length=dp.max_seq_len, padding='max_length', truncation='longest_first'), batched=True)

In [ ]:
idx = np.random.randint(len(begin_ds))
text = end_ds['text'][idx]
print(text)

word_attributions = cls_explainer(text)
print(cls_explainer.predicted_class_index, 
      cls_explainer.predicted_class_name
     )
cls_explainer.visualize("bert_viz.html")

import gc
gc.collect()
torch.cuda.empty_cache()

--- important links ---

https://github.com/cdpierse/transformers-interpret
  
https://shap.readthedocs.io/en/latest/example_notebooks/api_examples/plots/text.html